## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,17.79,82,92,1.95,overcast clouds
1,1,Hami,CN,42.80,93.45,8.39,26,3,3.34,clear sky
2,2,Landskrona,SE,55.87,12.83,10.01,68,70,5.68,broken clouds
3,3,Mahebourg,MU,-20.41,57.70,22.16,100,75,5.14,light rain
4,4,Ribeira Grande,PT,38.52,-28.70,21.42,90,100,10.47,overcast clouds


In [3]:
city_data_df = city_data_df.rename(columns={"Unnamed: 0":"City ID"})
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,17.79,82,92,1.95,overcast clouds
1,1,Hami,CN,42.80,93.45,8.39,26,3,3.34,clear sky
2,2,Landskrona,SE,55.87,12.83,10.01,68,70,5.68,broken clouds
3,3,Mahebourg,MU,-20.41,57.70,22.16,100,75,5.14,light rain
4,4,Ribeira Grande,PT,38.52,-28.70,21.42,90,100,10.47,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature for your vacation?"))
max_temp = float(input("What is the maximum temperature for your vacation?"))

What is the minimum temperature for your vacation?22
What is the maximum temperature for your vacation?25


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
new_city_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Mahebourg,MU,-20.41,57.70,22.16,100,75,5.14,light rain
8,8,Rikitea,PF,-23.12,-134.97,23.71,81,98,8.52,overcast clouds
11,11,Caravelas,BR,-17.71,-39.25,22.51,91,99,3.03,overcast clouds
13,13,Dingle,PH,11.00,122.67,24.63,94,93,4.20,light rain
17,17,Santiago Del Estero,AR,-27.80,-64.26,24.34,51,61,4.99,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
new_city_df.count()

City ID                95
City                   95
Country                93
Lat                    95
Lng                    95
Max Temp               95
Humidity               95
Cloudiness             95
Wind Speed             95
Current Description    95
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_city_df = new_city_df.dropna()
clean_city_df.count() # checked 

City ID                93
City                   93
Country                93
Lat                    93
Lng                    93
Max Temp               93
Humidity               93
Cloudiness             93
Wind Speed             93
Current Description    93
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mahebourg,MU,22.16,light rain,-20.41,57.70,
8,Rikitea,PF,23.71,overcast clouds,-23.12,-134.97,
11,Caravelas,BR,22.51,overcast clouds,-17.71,-39.25,
13,Dingle,PH,24.63,light rain,11.00,122.67,
17,Santiago Del Estero,AR,24.34,broken clouds,-27.80,-64.26,
23,Bambous Virieux,MU,22.14,light rain,-20.34,57.76,
29,Bousso,TD,22.63,overcast clouds,10.48,16.71,
39,Korhogo,CI,22.71,light rain,9.45,-5.63,
45,Pipri,IN,23.26,few clouds,19.80,75.53,
54,Karratha,AU,22.43,clear sky,-20.74,116.85,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location']=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found. Skipping")
        hotel_df.loc[index, "Hotel Name"] = ""

Hotel not found. Skipping
Hotel not found. Skipping
Hotel not found. Skipping
Hotel not found. Skipping
Hotel not found. Skipping
Hotel not found. Skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mahebourg,MU,22.16,light rain,-20.41,57.70,Shandrani Beachcomber Resort & Spa
8,Rikitea,PF,23.71,overcast clouds,-23.12,-134.97,People ThankYou
11,Caravelas,BR,22.51,overcast clouds,-17.71,-39.25,Pousada dos Navegantes
13,Dingle,PH,24.63,light rain,11.00,122.67,SEGAYA PROPERTY
17,Santiago Del Estero,AR,24.34,broken clouds,-27.80,-64.26,Apartamentos Castelli 90
23,Bambous Virieux,MU,22.14,light rain,-20.34,57.76,Casa Tia Villa
29,Bousso,TD,22.63,overcast clouds,10.48,16.71,
39,Korhogo,CI,22.71,light rain,9.45,-5.63,Hôtel Non Stop
45,Pipri,IN,23.26,few clouds,19.80,75.53,Ajay Patekar home
54,Karratha,AU,22.43,clear sky,-20.74,116.85,ibis Styles Karratha


In [11]:
df1 = hotel_df.replace('',np.nan, regex=True)
df1.count()

City                   93
Country                93
Max Temp               93
Current Description    93
Lat                    93
Lng                    93
Hotel Name             87
dtype: int64

In [12]:
clean_hotel_df = df1.dropna()
clean_hotel_df.count() # dropped NaN

City                   87
Country                87
Max Temp               87
Current Description    87
Lat                    87
Lng                    87
Hotel Name             87
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# from s.6.5.4
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig 

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# D2 completed.